# Data Processing

In this notebook you will find code for CSV processing. The raw CSV is formatted like this:

<img src="mRNA_raw_csv.png" width="400">

Column labels represent <font color = 'red'> <b> patients </b> </font> with either mild or severe RSV. Row labels represent <font color = 'red'> <b> genes </b> </font>. For each observation, we have a <font color = 'red'> <b> non-normalized mRNA count </b> </font> of that gene in each patient.

Let's start by importing our packages, then loading in our data.

In [18]:
# Imports
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load in data
pd.set_option('display.max_columns', None) # We'd like to be able to view all columns

mRNA_data = pd.read_csv("../RAW_DATA/mRNA_count_data.csv", encoding = "UTF-8") # mRNA data
mRNA_data.head(10) # Let's take a look at the first 10 rows.



,Gene,GSM4715942,GSM4715943,GSM4715947,GSM4715948,GSM4715951,GSM4715952,GSM4715953,GSM4715954,GSM4715956,GSM4715957,GSM4715961,GSM4715962,GSM4715963,GSM4715964,GSM4715965,GSM4715966,GSM4715968,GSM4715970,GSM4715971,GSM4715972,GSM4715973,GSM4715975,GSM4715977,GSM4715980,GSM4715986,GSM4715987,GSM4715988,GSM4715989,GSM4715990,GSM4715994,GSM4715995,GSM4715996,GSM4715997,GSM4715999,GSM4716002,GSM4716003,GSM4716004
0,DPM1,201,259,257,248,810,353,333,125,367,351,325,270,392,433,313,425,354,361,348,607,407,320,514,442,543,321,400,454,476,484,428,211,76,307,333,194,263
1,SCYL3,635,454,519,585,1323,1013,964,339,695,617,634,624,541,558,503,705,705,540,514,890,622,791,654,818,1343,899,1111,1213,1294,1059,1058,540,295,820,759,397,834
2,C1orf112,174,173,144,219,393,242,323,102,218,202,341,186,215,412,117,299,228,146,217,322,230,324,274,315,482,283,371,372,384,363,361,134,105,304,201,174,209
3,FGR,1925,4931,4142,4080,4505,11083,8909,3411,3504,5208,4884,7127,2956,2763,5713,4106,5247,8067,3317,12291,7987,7154,4805,6989,5308,6174,5757,6331,6466,7845,4402,14466,718,4426,8914,3905,6426
4,CFH,5,16,24,16,45,21,7,53,30,42,25,19,15,34,37,10,29,8,24,18,52,16,17,32,63,17,11,30,13,37,7,1,40,20,27,11,19
5,FUCA2,135,228,206,164,316,301,311,64,205,240,230,219,142,208,290,194,283,290,161,436,341,238,264,329,451,386,389,371,391,258,377,141,64,216,215,100,230
6,GCLC,254,258,175,224,539,551,363,148,215,266,460,1152,340,587,635,476,1650,362,401,677,920,492,579,1377,1040,424,462,361,614,464,444,187,250,256,241,344,354
7,NFYA,606,772,842,691,1773,1191,897,277,684,753,863,1091,984,913,843,949,1063,1120,1033,1933,1242,1006,849,1439,1368,706,892,1100,1181,1709,1185,805,305,930,731,642,919
8,STPG1,80,61,67,102,182,122,65,85,90,85,67,128,54,66,90,71,78,46,49,76,79,70,49,138,159,119,180,126,133,110,93,33,112,99,73,38,106
9,NIPAL3,1010,940,675,959,2466,2005,1089,180,757,1056,1069,1265,975,1011,959,1503,1610,977,735,1313,1486,1158,1168,1937,2466,1079,1855,1974,1986,1690,1924,507,522,1437,948,613,1405


First order of business - let's get rid of those gross column labels. For each column, we'd like to encode information about their gender, as well as whether they were a mild or severe case. We'll get that info from the patient data csv (0 = Female, 1 = Male)

In [19]:
patient_data = pd.read_csv("../RAW_DATA/patient_data.csv") # patient data
patient_data.head(40)

,Accession,Sex,Age,Batch,Hospital,WHO_LRTI,Severity
0,GSM4715942,0,1,0,0,2,mild
1,GSM4715943,0,0,0,0,1,mild
2,GSM4715947,0,1,0,0,4,severe
3,GSM4715948,1,0,0,0,2,mild
4,GSM4715951,0,0,1,0,1,mild
5,GSM4715952,1,0,1,0,1,mild
6,GSM4715953,1,1,1,0,2,mild
7,GSM4715954,1,3,1,0,4,severe
8,GSM4715956,0,2,1,0,4,severe
9,GSM4715957,0,1,1,0,1,mild


There are few enough patients that we can manually enter the info.

In [20]:
new_names = {"GSM4715942": "F_mild_1", "GSM4715943": "F_mild_2", "GSM4715947": "F_severe_1",
             "GSM4715948": "M_mild_3", "GSM4715951": "F_mild_4", "GSM4715952": "M_mild_5",
             "GSM4715953": "M_mild_6", "GSM4715954": "M_severe_2", "GSM4715956": "F_severe_3",
             "GSM4715957": "F_mild_7", "GSM4715961": "M_mild_8", "GSM4715962": "M_mild_9", 
             "GSM4715963": "M_mild_10", "GSM4715964": "F_mild_11", "GSM4715965": "F_mild_12",
             "GSM4715966": "M_mild_13", "GSM4715968": "F_mild_14", "GSM4715970": "M_mild_15",
             "GSM4715971": "F_mild_16", "GSM4715972": "M_severe_4", "GSM4715973": "M_mild_17",
             "GSM4715975": "F_severe_5", "GSM4715977": "M_mild_18", "GSM4715980": "F_mild_19",
             "GSM4715986": "M_mild_20", "GSM4715987": "M_mild_21", "GSM4715988": "F_mild_22",
             "GSM4715989": "M_mild_23", "GSM4715990": "M_mild_24", "GSM4715994": "F_mild_25",
             "GSM4715995": "F_mild_26", "GSM4715996": "F_severe_6", "GSM4715997": "F_mild_27",
             "GSM4715999": "M_mild_28", "GSM4716002": "M_severe_7", "GSM4716003": "F_severe_8",
             "GSM4716004": "M_mild_29"}

mRNA_data = mRNA_data.rename(columns = new_names)
mRNA_data.head(10)

,Gene,F_mild_1,F_mild_2,F_severe_1,M_mild_3,F_mild_4,M_mild_5,M_mild_6,M_severe_2,F_severe_3,F_mild_7,M_mild_8,M_mild_9,M_mild_10,F_mild_11,F_mild_12,M_mild_13,F_mild_14,M_mild_15,F_mild_16,M_severe_4,M_mild_17,F_severe_5,M_mild_18,F_mild_19,M_mild_20,M_mild_21,F_mild_22,M_mild_23,M_mild_24,F_mild_25,F_mild_26,F_severe_6,F_mild_27,M_mild_28,M_severe_7,F_severe_8,M_mild_29
0,DPM1,201,259,257,248,810,353,333,125,367,351,325,270,392,433,313,425,354,361,348,607,407,320,514,442,543,321,400,454,476,484,428,211,76,307,333,194,263
1,SCYL3,635,454,519,585,1323,1013,964,339,695,617,634,624,541,558,503,705,705,540,514,890,622,791,654,818,1343,899,1111,1213,1294,1059,1058,540,295,820,759,397,834
2,C1orf112,174,173,144,219,393,242,323,102,218,202,341,186,215,412,117,299,228,146,217,322,230,324,274,315,482,283,371,372,384,363,361,134,105,304,201,174,209
3,FGR,1925,4931,4142,4080,4505,11083,8909,3411,3504,5208,4884,7127,2956,2763,5713,4106,5247,8067,3317,12291,7987,7154,4805,6989,5308,6174,5757,6331,6466,7845,4402,14466,718,4426,8914,3905,6426
4,CFH,5,16,24,16,45,21,7,53,30,42,25,19,15,34,37,10,29,8,24,18,52,16,17,32,63,17,11,30,13,37,7,1,40,20,27,11,19
5,FUCA2,135,228,206,164,316,301,311,64,205,240,230,219,142,208,290,194,283,290,161,436,341,238,264,329,451,386,389,371,391,258,377,141,64,216,215,100,230
6,GCLC,254,258,175,224,539,551,363,148,215,266,460,1152,340,587,635,476,1650,362,401,677,920,492,579,1377,1040,424,462,361,614,464,444,187,250,256,241,344,354
7,NFYA,606,772,842,691,1773,1191,897,277,684,753,863,1091,984,913,843,949,1063,1120,1033,1933,1242,1006,849,1439,1368,706,892,1100,1181,1709,1185,805,305,930,731,642,919
8,STPG1,80,61,67,102,182,122,65,85,90,85,67,128,54,66,90,71,78,46,49,76,79,70,49,138,159,119,180,126,133,110,93,33,112,99,73,38,106
9,NIPAL3,1010,940,675,959,2466,2005,1089,180,757,1056,1069,1265,975,1011,959,1503,1610,977,735,1313,1486,1158,1168,1937,2466,1079,1855,1974,1986,1690,1924,507,522,1437,948,613,1405


Now let's rearrange these columns so that mild and severe patients are in separate blocs. 

In [21]:
mild_cases = []
severe_cases = []

# Generate new ordering of column labels, with mild ones listed first
for label in mRNA_data.columns.tolist():
    if label[2] == 'm':
        mild_cases.append(label)
    elif label[2] == 's':
        severe_cases.append(label)

        
mRNA_data = mRNA_data.reindex(columns = ["Gene"] + mild_cases + severe_cases)
mRNA_data.head(30)

,Gene,F_mild_1,F_mild_2,M_mild_3,F_mild_4,M_mild_5,M_mild_6,F_mild_7,M_mild_8,M_mild_9,M_mild_10,F_mild_11,F_mild_12,M_mild_13,F_mild_14,M_mild_15,F_mild_16,M_mild_17,M_mild_18,F_mild_19,M_mild_20,M_mild_21,F_mild_22,M_mild_23,M_mild_24,F_mild_25,F_mild_26,F_mild_27,M_mild_28,M_mild_29,F_severe_1,M_severe_2,F_severe_3,M_severe_4,F_severe_5,F_severe_6,M_severe_7,F_severe_8
0,DPM1,201,259,248,810,353,333,351,325,270,392,433,313,425,354,361,348,407,514,442,543,321,400,454,476,484,428,76,307,263,257,125,367,607,320,211,333,194
1,SCYL3,635,454,585,1323,1013,964,617,634,624,541,558,503,705,705,540,514,622,654,818,1343,899,1111,1213,1294,1059,1058,295,820,834,519,339,695,890,791,540,759,397
2,C1orf112,174,173,219,393,242,323,202,341,186,215,412,117,299,228,146,217,230,274,315,482,283,371,372,384,363,361,105,304,209,144,102,218,322,324,134,201,174
3,FGR,1925,4931,4080,4505,11083,8909,5208,4884,7127,2956,2763,5713,4106,5247,8067,3317,7987,4805,6989,5308,6174,5757,6331,6466,7845,4402,718,4426,6426,4142,3411,3504,12291,7154,14466,8914,3905
4,CFH,5,16,16,45,21,7,42,25,19,15,34,37,10,29,8,24,52,17,32,63,17,11,30,13,37,7,40,20,19,24,53,30,18,16,1,27,11
5,FUCA2,135,228,164,316,301,311,240,230,219,142,208,290,194,283,290,161,341,264,329,451,386,389,371,391,258,377,64,216,230,206,64,205,436,238,141,215,100
6,GCLC,254,258,224,539,551,363,266,460,1152,340,587,635,476,1650,362,401,920,579,1377,1040,424,462,361,614,464,444,250,256,354,175,148,215,677,492,187,241,344
7,NFYA,606,772,691,1773,1191,897,753,863,1091,984,913,843,949,1063,1120,1033,1242,849,1439,1368,706,892,1100,1181,1709,1185,305,930,919,842,277,684,1933,1006,805,731,642
8,STPG1,80,61,102,182,122,65,85,67,128,54,66,90,71,78,46,49,79,49,138,159,119,180,126,133,110,93,112,99,106,67,85,90,76,70,33,73,38
9,NIPAL3,1010,940,959,2466,2005,1089,1056,1069,1265,975,1011,959,1503,1610,977,735,1486,1168,1937,2466,1079,1855,1974,1986,1690,1924,522,1437,1405,675,180,757,1313,1158,507,948,613


Now let's pull in the data for our controls. We'll want to merge into our initial dataframe.

In [22]:
# Read in controls dataframe
controls_data = pd.read_csv("controls.csv")
print(controls_data.columns.tolist())
controls_data.head(10)

['Gene', 'Case 1', 'Case 2', 'Case 3', 'Case 4', 'Case 5', 'Case 6', 'Case 7', 'Case 8', 'Case 9', 'Case 10', 'Case 11', 'Case 12', 'Case 13', 'Case 14', 'Case 15', 'Case 16', 'Case 17', 'Case 18', 'Case 19', 'Case 20', 'Case 21', 'Case 22', 'Case 23', 'Case 24', 'Case 25', 'Case 26', 'Case 27', 'Case 28', 'Case 29', 'Case 30', 'Case 31', 'Case 32', 'Case 33', 'Case 34', 'Case 35', 'Case 36', 'Case 37', 'Case 38', 'Case 39', 'Case 40', 'Case 41', 'Case 42', 'Case 43', 'Case 44', 'Case 45', 'Case 46', 'Case 47', 'Case 48', 'Case 49', 'Case 50', 'Case 51', 'Case 52', 'Case 53', 'Case 54', 'Case 55', 'Case 56', 'Case 57', 'Case 58', 'Case 59', 'Case 60', 'Case 61', 'Case 62', 'Case 63', 'Case 64']


,Gene,Case 1,Case 2,Case 3,Case 4,Case 5,Case 6,Case 7,Case 8,Case 9,Case 10,Case 11,Case 12,Case 13,Case 14,Case 15,Case 16,Case 17,Case 18,Case 19,Case 20,Case 21,Case 22,Case 23,Case 24,Case 25,Case 26,Case 27,Case 28,Case 29,Case 30,Case 31,Case 32,Case 33,Case 34,Case 35,Case 36,Case 37,Case 38,Case 39,Case 40,Case 41,Case 42,Case 43,Case 44,Case 45,Case 46,Case 47,Case 48,Case 49,Case 50,Case 51,Case 52,Case 53,Case 54,Case 55,Case 56,Case 57,Case 58,Case 59,Case 60,Case 61,Case 62,Case 63,Case 64
0,TSPAN6,7,12,7,7,5,6,5,3,3,4,41,17,2,28,25,44,58,4,2,5,7,6,3,0,7,4,11,11,5,5,3,11,8,5,8,19,10,5,6,21,9,16,7,11,5,53,10,18,22,8,12,27,9,4,18,3,60,34,22,37,1,18,6,29
1,TNMD,0,1,0,0,0,0,0,0,0,0,0,0,0,8,13,10,13,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,4,2,0,0,0,19,1,0,2,0,1,0,0
2,DPM1,227,201,259,224,154,143,257,248,133,147,810,353,333,125,164,367,351,199,253,220,325,270,392,433,313,425,431,354,284,361,348,607,407,249,320,374,514,307,505,442,709,586,455,606,519,543,321,400,454,476,336,285,386,484,428,211,76,312,307,391,172,333,194,263
3,SCYL3,500,635,454,643,473,309,519,585,413,333,1323,1013,964,339,330,695,617,570,725,677,634,624,541,558,503,705,749,705,497,540,514,890,622,601,791,641,654,688,754,818,882,998,873,914,882,1343,899,1111,1213,1294,1026,567,993,1059,1058,540,295,892,820,1011,378,759,397,834
4,C1orf112,158,174,173,157,134,131,144,219,116,79,393,242,323,102,106,218,202,167,267,270,341,186,215,412,117,299,227,228,194,146,217,322,230,177,324,226,274,273,251,315,379,317,306,313,235,482,283,371,372,384,262,208,248,363,361,134,105,317,304,341,128,201,174,209
5,FGR,4723,1925,4931,10412,6132,3896,4142,4080,2408,3355,4505,11083,8909,3411,2203,3504,5208,7497,8619,4944,4884,7127,2956,2763,5713,4106,5945,5247,6825,8067,3317,12291,7987,9422,7154,5977,4805,5964,8811,6989,11269,8919,6361,10061,11220,5308,6174,5757,6331,6466,7375,2746,6846,7845,4402,14466,718,3665,4426,6031,5007,8914,3905,6426
6,CFH,7,5,16,9,27,11,24,16,2,3,45,21,7,53,41,30,42,26,18,11,25,19,15,34,37,10,14,29,10,8,24,18,52,19,16,14,17,19,10,32,23,8,87,15,17,63,17,11,30,13,14,26,40,37,7,1,40,30,20,18,8,27,11,19
7,FUCA2,158,135,228,176,195,170,206,164,108,140,316,301,311,64,53,205,240,150,174,145,230,219,142,208,290,194,333,283,175,290,161,436,341,281,238,194,264,268,267,329,377,278,275,360,363,451,386,389,371,391,440,195,310,258,377,141,64,245,216,314,191,215,100,230
8,GCLC,184,254,258,248,241,179,175,224,269,288,539,551,363,148,110,215,266,707,406,894,460,1152,340,587,635,476,666,1650,886,362,401,677,920,690,492,916,579,496,404,1377,776,847,722,1306,544,1040,424,462,361,614,380,170,243,464,444,187,250,448,256,298,281,241,344,354
9,NFYA,616,606,772,1001,769,438,842,691,786,883,1773,1191,897,277,372,684,753,1384,1401,1071,863,1091,984,913,843,949,1065,1063,1212,1120,1033,1933,1242,822,1006,1137,849,947,1449,1439,1220,1836,1551,1268,1991,1368,706,892,1100,1181,797,515,1217,1709,1185,805,305,1095,930,1081,492,731,642,919


In [23]:
print(mRNA_data.columns.tolist())
mRNA_data = mRNA_data.merge(controls_data, on = "Gene")
mRNA_data.head(20)

['Gene', 'F_mild_1', 'F_mild_2', 'M_mild_3', 'F_mild_4', 'M_mild_5', 'M_mild_6', 'F_mild_7', 'M_mild_8', 'M_mild_9', 'M_mild_10', 'F_mild_11', 'F_mild_12', 'M_mild_13', 'F_mild_14', 'M_mild_15', 'F_mild_16', 'M_mild_17', 'M_mild_18', 'F_mild_19', 'M_mild_20', 'M_mild_21', 'F_mild_22', 'M_mild_23', 'M_mild_24', 'F_mild_25', 'F_mild_26', 'F_mild_27', 'M_mild_28', 'M_mild_29', 'F_severe_1', 'M_severe_2', 'F_severe_3', 'M_severe_4', 'F_severe_5', 'F_severe_6', 'M_severe_7', 'F_severe_8']


,Gene,F_mild_1,F_mild_2,M_mild_3,F_mild_4,M_mild_5,M_mild_6,F_mild_7,M_mild_8,M_mild_9,M_mild_10,F_mild_11,F_mild_12,M_mild_13,F_mild_14,M_mild_15,F_mild_16,M_mild_17,M_mild_18,F_mild_19,M_mild_20,M_mild_21,F_mild_22,M_mild_23,M_mild_24,F_mild_25,F_mild_26,F_mild_27,M_mild_28,M_mild_29,F_severe_1,M_severe_2,F_severe_3,M_severe_4,F_severe_5,F_severe_6,M_severe_7,F_severe_8,Case 1,Case 2,Case 3,Case 4,Case 5,Case 6,Case 7,Case 8,Case 9,Case 10,Case 11,Case 12,Case 13,Case 14,Case 15,Case 16,Case 17,Case 18,Case 19,Case 20,Case 21,Case 22,Case 23,Case 24,Case 25,Case 26,Case 27,Case 28,Case 29,Case 30,Case 31,Case 32,Case 33,Case 34,Case 35,Case 36,Case 37,Case 38,Case 39,Case 40,Case 41,Case 42,Case 43,Case 44,Case 45,Case 46,Case 47,Case 48,Case 49,Case 50,Case 51,Case 52,Case 53,Case 54,Case 55,Case 56,Case 57,Case 58,Case 59,Case 60,Case 61,Case 62,Case 63,Case 64
0,DPM1,201,259,248,810,353,333,351,325,270,392,433,313,425,354,361,348,407,514,442,543,321,400,454,476,484,428,76,307,263,257,125,367,607,320,211,333,194,227,201,259,224,154,143,257,248,133,147,810,353,333,125,164,367,351,199,253,220,325,270,392,433,313,425,431,354,284,361,348,607,407,249,320,374,514,307,505,442,709,586,455,606,519,543,321,400,454,476,336,285,386,484,428,211,76,312,307,391,172,333,194,263
1,SCYL3,635,454,585,1323,1013,964,617,634,624,541,558,503,705,705,540,514,622,654,818,1343,899,1111,1213,1294,1059,1058,295,820,834,519,339,695,890,791,540,759,397,500,635,454,643,473,309,519,585,413,333,1323,1013,964,339,330,695,617,570,725,677,634,624,541,558,503,705,749,705,497,540,514,890,622,601,791,641,654,688,754,818,882,998,873,914,882,1343,899,1111,1213,1294,1026,567,993,1059,1058,540,295,892,820,1011,378,759,397,834
2,C1orf112,174,173,219,393,242,323,202,341,186,215,412,117,299,228,146,217,230,274,315,482,283,371,372,384,363,361,105,304,209,144,102,218,322,324,134,201,174,158,174,173,157,134,131,144,219,116,79,393,242,323,102,106,218,202,167,267,270,341,186,215,412,117,299,227,228,194,146,217,322,230,177,324,226,274,273,251,315,379,317,306,313,235,482,283,371,372,384,262,208,248,363,361,134,105,317,304,341,128,201,174,209
3,FGR,1925,4931,4080,4505,11083,8909,5208,4884,7127,2956,2763,5713,4106,5247,8067,3317,7987,4805,6989,5308,6174,5757,6331,6466,7845,4402,718,4426,6426,4142,3411,3504,12291,7154,14466,8914,3905,4723,1925,4931,10412,6132,3896,4142,4080,2408,3355,4505,11083,8909,3411,2203,3504,5208,7497,8619,4944,4884,7127,2956,2763,5713,4106,5945,5247,6825,8067,3317,12291,7987,9422,7154,5977,4805,5964,8811,6989,11269,8919,6361,10061,11220,5308,6174,5757,6331,6466,7375,2746,6846,7845,4402,14466,718,3665,4426,6031,5007,8914,3905,6426
4,CFH,5,16,16,45,21,7,42,25,19,15,34,37,10,29,8,24,52,17,32,63,17,11,30,13,37,7,40,20,19,24,53,30,18,16,1,27,11,7,5,16,9,27,11,24,16,2,3,45,21,7,53,41,30,42,26,18,11,25,19,15,34,37,10,14,29,10,8,24,18,52,19,16,14,17,19,10,32,23,8,87,15,17,63,17,11,30,13,14,26,40,37,7,1,40,30,20,18,8,27,11,19
5,FUCA2,135,228,164,316,301,311,240,230,219,142,208,290,194,283,290,161,341,264,329,451,386,389,371,391,258,377,64,216,230,206,64,205,436,238,141,215,100,158,135,228,176,195,170,206,164,108,140,316,301,311,64,53,205,240,150,174,145,230,219,142,208,290,194,333,283,175,290,161,436,341,281,238,194,264,268,267,329,377,278,275,360,363,451,386,389,371,391,440,195,310,258,377,141,64,245,216,314,191,215,100,230
6,GCLC,254,258,224,539,551,363,266,460,1152,340,587,635,476,1650,362,401,920,579,1377,1040,424,462,361,614,464,444,250,256,354,175,148,215,677,492,187,241,344,184,254,258,248,241,179,175,224,269,288,539,551,363,148,110,215,266,707,406,894,460,1152,340,587,635,476,666,1650,886,362,401,677,920,690,492,916,579,496,404,1377,776,847,722,1306,544,1040,424,462,361,614,380,170,243,464,444,187,250,448,256,298,281,241,344,354
7,NFYA,606,772,691,1773,1191,897,753,863,1091,984,913,843,949,1063,1120,1033,1242,849,1439,1368,706,892,1100,1181,1709,1185,305,930,919,842,277,684,1933,1006,805,731,642,616,606,772,1001,769,438,842,691,786,883,1773,1191,897,277,3

Great! Now, our next order of business is to normalize each column by total patient mRNA.

In [24]:
# Calculate sum of each column, convert to proportions. 

def logarithm(element):
    if element > 0:
        return math.log(element)
    else:
        return 0

# Normalize by total mRNA content per patient.
for label in mRNA_data.columns.tolist()[1:]:
    mRNA_data[label] = mRNA_data[label].astype(int)
    col_sum = mRNA_data[label].sum()
    mRNA_data[label] = (mRNA_data[label].div(col_sum))
    mRNA_data[label] = mRNA_data[label].apply(logarithm) # Take log proportion to avoid really small numbers.

# Let's take a look again.
mRNA_data.head(10)

,Gene,F_mild_1,F_mild_2,M_mild_3,F_mild_4,M_mild_5,M_mild_6,F_mild_7,M_mild_8,M_mild_9,M_mild_10,F_mild_11,F_mild_12,M_mild_13,F_mild_14,M_mild_15,F_mild_16,M_mild_17,M_mild_18,F_mild_19,M_mild_20,M_mild_21,F_mild_22,M_mild_23,M_mild_24,F_mild_25,F_mild_26,F_mild_27,M_mild_28,M_mild_29,F_severe_1,M_severe_2,F_severe_3,M_severe_4,F_severe_5,F_severe_6,M_severe_7,F_severe_8,Case 1,Case 2,Case 3,Case 4,Case 5,Case 6,Case 7,Case 8,Case 9,Case 10,Case 11,Case 12,Case 13,Case 14,Case 15,Case 16,Case 17,Case 18,Case 19,Case 20,Case 21,Case 22,Case 23,Case 24,Case 25,Case 26,Case 27,Case 28,Case 29,Case 30,Case 31,Case 32,Case 33,Case 34,Case 35,Case 36,Case 37,Case 38,Case 39,Case 40,Case 41,Case 42,Case 43,Case 44,Case 45,Case 46,Case 47,Case 48,Case 49,Case 50,Case 51,Case 52,Case 53,Case 54,Case 55,Case 56,Case 57,Case 58,Case 59,Case 60,Case 61,Case 62,Case 63,Case 64
0,DPM1,-10.495318,-10.537266,-10.411985,-10.035166,-10.775418,-10.763280,-10.332380,-10.431805,-10.781632,-10.144019,-10.213861,-10.507874,-10.234819,-10.608586,-10.517203,-10.325758,-10.556518,-10.090653,-10.552841,-10.438853,-10.598390,-10.564971,-10.500881,-10.529793,-10.509058,-10.479612,-10.824071,-10.540797,-10.719486,-10.647084,-10.593963,-10.075558,-10.441338,-10.574109,-11.040751,-10.552491,-10.653034,-10.573358,-10.494605,-10.536768,-11.076896,-11.094730,-10.746240,-10.646705,-10.411492,-10.773244,-10.804425,-10.034403,-10.774905,-10.762649,-10.593538,-10.280745,-10.074660,-10.331640,-11.051653,-10.899309,-10.815565,-10.431016,-10.781135,-10.143308,-10.213000,-10.507164,-10.234045,-10.496478,-10.608011,-10.668865,-10.516590,-10.325073,-10.440802,-10.555987,-10.801691,-10.573476,-10.560253,-10.089866,-10.586054,-10.382640,-10.552261,-10.214060,-10.441881,-10.444915,-10.376679,-10.628495,-10.438010,-10.597669,-10.564167,-10.500057,-10.529017,-10.678219,-10.247863,-10.640927,-10.508425,-10.478880,-11.040543,-10.823269,-10.646576,-10.540090,-10.436501,-10.875967,-10.551842,-10.652710,-10.718914
1,SCYL3,-9.344997,-9.975997,-9.553802,-9.544543,-9.721215,-9.700331,-9.768298,-9.763581,-9.943903,-9.821862,-9.960240,-10.033487,-9.728711,-9.919686,-10.114512,-9.935737,-10.132391,-9.849768,-9.937288,-9.533301,-9.568548,-9.543419,-9.518126,-9.529717,-9.726063,-9.574600,-9.467829,-9.558340,-9.565407,-9.944256,-9.596277,-9.437008,-10.058645,-9.669132,-10.101040,-9.728632,-9.936955,-9.783700,-9.344285,-9.975499,-10.022397,-9.972587,-9.975744,-9.943877,-9.553309,-9.640145,-9.986715,-9.543780,-9.720702,-9.699700,-9.595852,-9.581518,-9.436110,-9.767557,-9.999321,-9.846527,-9.691521,-9.762792,-9.943407,-9.821151,-9.959379,-10.032777,-9.727936,-9.943847,-9.919110,-10.109249,-10.113899,-9.935052,-10.058109,-10.131860,-9.920549,-9.668499,-10.021479,-9.848982,-9.779113,-9.981807,-9.936708,-9.995723,-9.909448,-9.793277,-9.965729,-10.098207,-9.532458,-9.567827,-9.542616,-9.517302,-9.528941,-9.561907,-9.559993,-9.696034,-9.725429,-9.573867,-10.100832,-9.467027,-9.596113,-9.557634,-9.486513,-10.088568,-9.727983,-9.936632,-9.564835
2,C1orf112,-10.639567,-10.940803,-10.536342,-10.758391,-11.152948,-10.793770,-10.884899,-10.383748,-11.154307,-10.744643,-10.263576,-11.491904,-10.586465,-11.048538,-11.422475,-10.798063,-11.127252,-10.719748,-10.891578,-10.558018,-10.724384,-10.640233,-10.700084,-10.744568,-10.796740,-10.649858,-10.500844,-10.550617,-10.949306,-11.226347,-10.797304,-10.596425,-11.075315,-10.561687,-11.494769,-11.057329,-10.761836,-10.935713,-10.638854,-10.940304,-11.432296,-11.233843,-10.833888,-11.225967,-10.535849,-10.910003,-11.425410,-10.757628,-11.152435,-10.793139,-10.796879,-10.717172,-10.595526,-10.884158,-11.226964,-10.845450,-10.610770,-10.382959,-11.153810,-10.743932,-10.262714,-11.491193,-10.585691,-11.137636,-11.047962,-11.049981,-11.421861,-10.797378,-11.074779,-11.126721,-11.142995,-10.561054,-11.063974,-10.718961,-10.703430,-11.081746,-10.890998,-10.840379,-11.056300,-10.841627,-11.037356,-11.420814,-10.557175,-10.723664,-10.639429,-10.699260,-10.743792,-10

Perfect! Let's write this into a CSV.

In [25]:
mRNA_data.to_csv("normalized_mRNA_counts.csv", index = False)